In [1]:
from keras import applications
from keras import models
from keras import layers
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense, Input
from keras.layers.normalization import BatchNormalization
from keras import regularizers
import matplotlib.pyplot as plt
from keras.models import load_model
from keras.utils import layer_utils, np_utils
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.datasets import make_classification
from sklearn.preprocessing import label_binarize
from scipy import interp
from itertools import cycle
from sklearn.metrics import roc_curve, auc
from keras.models import Model
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras import backend as K

Using TensorFlow backend.


In [2]:
#Ensembling ConvNets using Keras
image_width = 224
image_height = 224

train_dir = "D:/retinal_samples/basian_opt_testing/new_new_train/augmented_224/train"
validation_dir = "D:/retinal_samples/basian_opt_testing/new_valid"
    
image_width = 224
image_height = 224
batch_size = 16

In [3]:
def dataGen():
    train_datagen = ImageDataGenerator(rescale=1./255)
    validation_datagen = ImageDataGenerator(rescale=1./255)
    
    train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(image_width, image_height),
        batch_size=batch_size,
        shuffle=True,
        class_mode='categorical')
    
    validation_generator = validation_datagen.flow_from_directory(
            validation_dir,
            target_size=(image_width, image_height),
            batch_size=batch_size,
            class_mode='categorical',
            shuffle=False)
    
    return train_generator, validation_generator

In [5]:
def getXValYValSplit(generator):
    x_val, y_val = zip(*(generator[i] for i in range(len(generator))))
    x_val, y_val = np.vstack(x_val), np.vstack(y_val)
    
    return x_val, y_val

In [ ]:
def evaluate_error(model, x_test):
    pred = model.predict(x_test, batch_size = batch_size)
    pred = np.argmax(pred, axis=1)
    pred = np.expand_dims(pred, axis=1) # make same shape as y_test
    error = np.sum(np.not_equal(pred, y_test)) / y_test.shape[0]  
  
    return error

In [ ]:
def ensemble(models, model_input):
    outputs = [model.outputs[0] for model in models]
    y = Average()(outputs)
    
    model = Model(model_input, y, name='ensemble')
    
    return model

In [ ]:
train_generator, validation_generator = dataGen()

In [6]:
x_train, y_train = getXValYValSplit(train_generator)

In [7]:
#model_input = Input(shape=(224,224,3))
input_shape = x_train[0,:,:,:].shape
model_input = Input(shape=input_shape)

(224, 224, 3)

In [ ]:
vgg_model = load_model('D:/retinal_data_set_visioncare/models/vgg16/vgg16_dr.h5')
resnet_model = load_model('D:/retinal_data_set_visioncare/models/vgg16/vgg16_dr.h5')
densenet_model = load_model('D:/retinal_data_set_visioncare/models/vgg16/vgg16_dr.h5')

In [ ]:
models = [vgg_model, resnet_model, densenet_model]

In [ ]:
ensemble_model = ensemble(models, model_input)
# Save the model
ensemble_model.save('D:/retinal_data_set_visioncare/models/ensemble/ensemble_dr.h5')
evaluate_error(ensemble_model)